In [ ]:
import requests
import datetime
import pandas as pd
import json
import boto3
import http.client

def lambda_handler(event, context):
    import json
    import http.client
    
    # Replace with your actual API key from API-Football
    api_key = "30548bfbf8msh1742529d7115018p1ffd33jsnb2403e22cd90"
    league_id = 140
    season = '2023'
    
    # Set up the connection with the correct base URL
    conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")
    
    # Set up the headers correctly
    headers = {
        'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
        'x-rapidapi-key': api_key
    }
    
    # Make the request to get the standings for La Liga in the 2023 season
    conn.request("GET", f"/v3/standings?league={league_id}&season={season}", headers=headers)
    
    # Get the response
    res = conn.getresponse()
    data = res.read()
    
    data_json = json.loads(data.decode("utf-8"))
    standings = data_json['response'][0]['league']['standings'][0]
    
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M")
    
    # Upload to S3
    s3_bucket_name = "dwlprojectbucket"
    s3 = boto3.client('s3')
    
    # Upload standings to S3 with public read permissions
    s3_key = f"{formatted_datetime} - standings.json"
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(standings), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload standings data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload standings data to S3')
        }
    
    api_key = "744a3b85a26ce36af4604f139ec46b9c"
    state_code = ""
    country_code = "ES"
    
    data = {
        "Football Club Name": {
            "Real Madrid": "Madrid",
            "Barcelona": "Barcelona",
            "Girona": "Girona",
            "Atletico Madrid": "Madrid",
            "Athletic Club": "Bilbao",
            "Real Sociedad": "San Sebastian",
            "Real Betis": "Sevilla",
            "Villarreal": "Villarreal",
            "Valencia": "Valencia",
            "Getafe": "Getafe",
            "Alaves": "Vitoria-Gasteiz",
            "Sevilla": "Sevilla",
            "Osasuna": "Pamplona",
            "Las Palmas": "Las Palmas de Gran Canaria",
            "Rayo Vallecano": "Puente de Vallecas",
            "Celta Vigo": "Vigo",
            "Mallorca": "Palma de Mallorca",
            "Cadiz": "Cadiz",
            "Granada CF": "Granada",
            "Almeria": "Almeria"
        }
    }
    
    df = pd.DataFrame(data).reset_index()
    df.columns = ["Football Club Name", "City Name"]
    cities = df['City Name'].drop_duplicates()
    
    prediction_16_days = []
    for city_name in cities:
        url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name},{state_code},{country_code}&limit={1}&appid={api_key}"
        response = requests.get(url)
        data = response.json()
        latitude = data[0]['lat']
        longitude = data[0]['lon']
    
        url = f"http://api.openweathermap.org/data/2.5/forecast/daily?lat={latitude}&lon={longitude}&cnt={16}&appid={api_key}"
        response = requests.get(url)
        data = response.json()
        prediction_16_days.append(data)

    
    # Upload to S3
    s3_bucket_name = "dwlprojectbucket"
    s3_key = f"{formatted_datetime} - predictions16days.json"
    
    try:
        s3.put_object(Bucket=s3_bucket_name, Key=s3_key, Body=json.dumps(prediction_16_days), ACL='public-read')
    except Exception as e:
        print(f"Failed to upload predictions16days data to S3: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('Failed to upload predictions16days data to S3')
        }
    
    return {
        'statusCode': 200,
        'body': json.dumps('Data successfully uploaded to S3 with public read permissions')
    }
